In [100]:
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import os

from collections import defaultdict
from itertools import tee
from datetime import datetime, timedelta
from base64 import b64encode
import urllib
import sys
import logging

import os
import subprocess

from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import webbrowser
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

In [2]:
base_url = 'https://admin.psylio.com/assistance-requests'

try:
    logger.removeHandler(handler)
except NameError:
    pass
    
fmt = '[{asctime}] [{levelname}]  {funcName}() {message}'
formatter = logging.Formatter(fmt, style='{', datefmt='%Y-%m-%d %H:%M:%S')

handler = logging.StreamHandler()
handler.setFormatter(formatter)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(handler)

logger.debug("lala")

[2022-03-17 13:14:22] [DEBUG]  <module>() lala


In [3]:
def open_html(template, filename='tmp.html'):
    import webbrowser
    import os
  
    with open(filename, 'wb') as f:
        # writing the code into the file
        f.write(template)
      
    # 1st method how to open html files in chrome using
    filename = f'file:///{os.getcwd()}/{filename}'
    webbrowser.open_new_tab(filename)

In [4]:
def login(email, password, base_url='https://admin.psylio.com'):
    session = requests.Session()
    resp = session.get(base_url)
    
    soup = BeautifulSoup(resp.content, 'html.parser')
    _token = soup.find('input', {'name': '_token'}).get('value')
    
    payload = {
        '_token': _token,
        'login': {
            'email': email,
            'password': password,
            'account': 'ca',
        },
    }
    
    # print(json.dumps(payload))
   
    session.headers.update({
        'Content-Type': 'application/json',
        # 'origin': 'https://admin.psylio.com',
    })
    
    resp = session.post(f'{base_url}/login', data=json.dumps(payload))
    assert resp.status_code == 200, 'Login failed :('
    # open_html(resp.content)
    
    return session

In [5]:
email = '<email>'
password = '<password>'

In [235]:
print('Attempting login...', end=' ')
session = login(email, password)
print('Login successful!')

# set headers
session.headers.update({
    'Origin': 'https://admin.psylio.com',
    'Referer': 'https://admin.psylio.com/assistance-requests',
    'Sec-Fetch-Site': 'same-origin',
    'X-Requested-With': 'XMLHttpRequest',

    'Access-Control-Allow-Origin': 'https://admin.psylio.com',
    'Access-Control-Allow-Methods': 'POST',
    'Access-Control-Allow-Headers': 'X-Requested-With',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',


    'Sec-CH-UA': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'Sec-CH-UA-Mobile': '?0',
    'Sec-CH-UA-Platform': '"Linux"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
})

Attempting login... Login successful!


In [240]:
def get_records(session):
    
    def helper(endpoint):
        resp = session.get(endpoint)
        soup = BeautifulSoup(resp.content, 'html.parser')
        tbody = soup.find('table').tbody

        record_ids = []
        for row in tbody.find_all('tr'):
            cells = row.find_all('td')
            rid = cells[1].a.get('href').split('/')[4]
            record_ids.append(rid)

        records_df = pd.read_html(resp.text)[0]
        records_df[['Client 2', 'Courriel 2']] = ''

        records_df['record_id'] = record_ids
        records_df.set_index('record_id', inplace=True)

        for record_id, record in records_df.iterrows():
            profile_url = f'{base_url}/{record_id}/profile'
            resp = session.get(profile_url)

            soup = BeautifulSoup(resp.content, 'html.parser')
            clients = soup.find('div', {'class' : 'tab-content profile-informations clearfix'})

            for i, client in enumerate(clients.find_all('div', {'role': 'tabpanel'})):
                person_link = client.find('a').get('href')
                resp = session.get(person_link)

                soup = BeautifulSoup(resp.content, 'html.parser')

                fname = soup.find('input', {'id': 'firstname'}).get('value')
                lname = soup.find('input', {'id': 'lastname'}).get('value')
                email = soup.find('input', {'id': 'email'}).get('value')

                records_df.loc[record_id, f'Client {i + 1}'] = f'{lname}, {fname}'
                records_df.loc[record_id, f'Courriel {i + 1}'] = email

        return records_df
    
    
    base_url = 'https://admin.psylio.com/assistance-requests'
    
    active_records = helper(base_url)
    archived_records = helper(f'{base_url}/archive')
    
    active_records['Statut'] = 'Actif'
    archived_records['Statut'] = 'Archivé'
    
    records_df = pd.concat([active_records, archived_records])
    
    columns = ['Numéro de dossier', 'Statut', 'Client 1', 'Courriel 1', 'Client 2', 'Courriel 2']
    records_df = records_df[columns]
    
    return records_df

In [241]:
records_df = get_records(session)
records_df

,Numéro de dossier,Statut,Client 1,Courriel 1,Client 2,Courriel 2
record_id,,,,,,
2927e24b-cd71-444c-b015-b713cd23aeab,150,Actif,"blo, mymy",fdl9044@gmail.com,"blu, ladu",francis.de.ladurantaye@gmail.com
93b04553-0e8d-49ec-9b1c-179535a7628b,140,Actif,"blu, bli",fdl9044@gmail.com,,
4dabe765-b2a0-47b3-b969-0a64e562d5e1,170,Actif,"mou, caca",fdl9044@gmail.com,"lala, ohoh",francis.de.ladurantaye@gmail.com
7d2510c7-1614-4c02-8102-b9c7cf8479d4,160,Archivé,"toi, salut",fdl9044@gmail.com,,
5441a179-4ce1-42ab-bc77-858af619e5ae,100,Archivé,"supsup, supsup",fdl9044@gmail.com,,
8c438179-5cfe-45a3-a68f-b7ce45efae91,123,Archivé,"supsup, spuer",fdl9044@gmail.com,,
a75af7ea-c86a-4895-8913-f1a47202f105,100,Archivé,"lala, lala",francis.de.ladurantaye@gmail.com,,
1fe02096-dd22-435a-9cdb-b4877d933f12,111,Archivé,"bli#, blu#",fdl9044@gmail.com,,


In [242]:
def get_appointments(session, records_df):
    end = datetime.now()
    start = end - timedelta(days=30)
    
    endpoint = f'https://admin.psylio.com/appointments/agenda/calendar?start={start.date()}&end={end.date()}'
    resp = session.get(endpoint)
    
    columns = {
        'startDate': 'Date',
        'startHour': 'Heure début',
        'endHour': 'Heure fin',
    }
        
    appointments = []
    for appoint in resp.json():
        appoint = appoint['modal']['appointment']
        record_id = appoint['assistanceRequest']['data']['id']
        
        infos = {col: appoint[key] for key, col in columns.items()}
        infos['record_id'] = record_id
        
        appointments.append(infos)
    
    new_index = ['record_id', 'Date']
    appoints_df = pd.DataFrame(appointments).sort_values(by=new_index, ascending=False)
    appoints_df.set_index(new_index, inplace=True)
    
    appoints_df = appoints_df.join(records_df, on='record_id')
    
    return appoints_df

In [243]:
appoints_df = get_appointments(session, records_df)
appoints_df

Heure début Heure fin  \
record_id                            Date                               
93b04553-0e8d-49ec-9b1c-179535a7628b 2022-03-14       12:30     13:30   
4dabe765-b2a0-47b3-b969-0a64e562d5e1 2022-03-17       15:30     16:30   
                                     2022-03-16       15:30     16:30   
                                     2022-03-15       15:30     16:30   
                                     2022-03-14       15:30     16:30   
                                     2022-03-11       15:30     16:30   
                                     2022-03-10       15:30     16:30   
                                     2022-03-09       15:30     16:30   
                                     2022-03-08       15:30     16:30   
2927e24b-cd71-444c-b015-b713cd23aeab 2022-03-17       17:00     18:00   
                                     2022-03-16       17:00     18:00   
                                     2022-03-15       17:00     18:00   
                                     2022-03-14       09:00     10:00   
                                     2022-03-14       17:00     18:00   
                                     2022-03-07       09:00     10:00   

                                                 Numéro de dossier Statut  \
record_id                            Date                                   
93b04553-0e8d-49ec-9b1c-179535a7628b 2022-03-14                140  Actif   
4dabe765-b2a0-47b3-b969-0a64e562d5e1 2022-03-17                170  Actif   
                                     2022-03-16                170  Actif   
                                     2022-03-15                170  Actif   
                                     2022-03-14                170  Actif   
                                     2022-03-11                170  Actif   
                                     2022-03-10                170  Actif   
                                     2022-03-09                170  Actif   
                                     2022-03-08                170  Actif   
2927e24b-cd71-444c-b015-b713cd23aeab 2022-03-17                150  Actif   
                                     2022-03-16                150  Actif   
                                     2022-03-15                150  Actif   
                                     2022-03-14                150  Actif   
                                     2022-03-14                150  Actif   
                                     2022-03-07                150  Actif   

                                                  Client 1         Courriel 1  \
record_id                            Date                                       
93b04553-0e8d-49ec-9b1c-179535a7628b 2022-03-14   blu, bli  fdl9044@gmail.com   
4dabe765-b2a0-47b3-b969-0a64e562d5e1 2022-03-17  mou, caca  fdl9044@gmail.com   
                                     2022-03-16  mou, caca  fdl9044@gmail.com   
                                     2022-03-15  mou, caca  fdl9044@gmail.com   
                                     2022-03-14  mou, caca  fdl9044@gmail.com   
                                     2022-03-11  mou, caca  fdl9044@gmail.com   
                                     2022-03-10  mou, caca  fdl9044@gmail.com   
                                     2022-03-09  mou, caca  fdl9044@gmail.com   
                                     2022-03-08  mou, caca  fdl9044@gmail.com   
2927e24b-cd71-444c-b015-b713cd23aeab 2022-03-17  blo, mymy  fdl9044@gmail.com   
                                     2022-03-16  blo, mymy  fdl9044@gmail.com   
                                     2022-03-15  blo, mymy  fdl9044@gmail.com   
                                     2022-03-14  blo, mymy  fdl9044@gmail.com   
                                     2022-03-14  blo, mymy  fdl9044@gmail.com   
                                     2022-03-07  blo, mymy  fdl9044@gmail.com   

                                                   Client 2  \
record_id                            Date              

In [244]:
def get_invoices(session, records_df):
    all_invoices = []
    for record_id, _ in records_df.iterrows():
        record_invoices = get_invoices_for_record(session, record_id)
        all_invoices.append(record_invoices)
    
    invoices_df = pd.concat(all_invoices)
    
    invoices_df.rename(columns={'Facturé le': 'Date'}, inplace=True)
    
    new_index = ['record_id', 'Date']
    invoices_df.set_index(new_index, inplace=True)
    
    columns = ['invoice_id', 'Facture', 'Service(s)', 'État', 'Montant dû', 'Montant payé']
    invoices_df = invoices_df[columns]
    
    return invoices_df


def get_invoices_for_record(session, record_id):
    
    def helper(endpoint):
        try:
            resp = session.get(endpoint)
            soup = BeautifulSoup(resp.content, 'html.parser')
            tbody = soup.find('table').tbody

            invoice_ids = []
            for row in tbody.find_all('tr'):
                cells = row.find_all('td')
                iid = cells[0].a.get('href').split('/')[6]
                invoice_ids.append(iid)

            invoices_df = pd.read_html(resp.text)[0]
            invoices_df['invoice_id'] = invoice_ids
            invoices_df['record_id'] = record_id
        except AttributeError:
            invoices_df = pd.DataFrame()

        return invoices_df
    
    endpoint = f'{base_url}/{record_id}/invoices'
    
    unpaid_df = helper(endpoint)
    paid_df = helper(f'{endpoint}?state=paid')
    
    return pd.concat([unpaid_df, paid_df])

In [245]:
invoices_df = get_invoices(session, records_df)
invoices_df

invoice_id  \
record_id                            Date                                               
2927e24b-cd71-444c-b015-b713cd23aeab 2022-03-17  8e6a476c-bd15-4303-8195-be2f2553dbde   
                                     2022-03-16  db2d04a0-40c7-49e6-8acc-71fce8d55a65   
                                     2022-03-15  bc2f322c-6d69-41d2-a19d-8c8eda4d1e0d   
                                     2022-03-14  debc4c12-1304-4f98-b274-0d1a2ef30ae3   
                                     2022-03-07  accfaac4-65d2-4a51-9140-309912074747   
...                                                                               ...   
1fe02096-dd22-435a-9cdb-b4877d933f12 2022-03-14  c8ff92b4-eaf2-4b90-a319-8dc91c01c571   
                                     2022-03-14  f6696acc-4691-4126-9460-dd4924f6f127   
                                     2022-03-14  95de6856-9743-4bbd-a559-851832a8577c   
                                     2022-03-14  d060dc91-8fa4-45bb-915b-6a12d2f04357   
                                     2022-03-13  ba40563c-6890-424a-8622-9f5f924aad32   

                                                         Facture  \
record_id                            Date                          
2927e24b-cd71-444c-b015-b713cd23aeab 2022-03-17  INV-20220317-88   
                                     2022-03-16  INV-20220316-84   
                                     2022-03-15  INV-20220315-83   
                                     2022-03-14  INV-20220314-49   
                                     2022-03-07  INV-20220314-48   
...                                                          ...   
1fe02096-dd22-435a-9cdb-b4877d933f12 2022-03-14  INV-20220314-24   
                                     2022-03-14  INV-20220314-23   
                                     2022-03-14  INV-20220314-22   
                                     2022-03-14  INV-20220314-20   
                                     2022-03-13  INV-20220313-19   

                                                   Service(s)         État  \
record_id                            Date                                    
2927e24b-cd71-444c-b015-b713cd23aeab 2022-03-17  Suivi client    Brouillon   
                                     2022-03-16  Suivi client  Reçu envoyé   
                                     2022-03-15  Suivi client        Payée   
                                     2022-03-14  Suivi client  Reçu envoyé   
                                     2022-03-07  Suivi client  Reçu envoyé   
...                                                       ...          ...   
1fe02096-dd22-435a-9cdb-b4877d933f12 2022-03-14  Suivi client        Payée   
                                     2022-03-14  Suivi client        Payée   
                                     2022-03-14  Suivi client        Payée   
                                     2022-03-14  Suivi client  Reçu envoyé   
                                     2022-03-13  Suivi client  Reçu envoyé   

                                                Montant dû Montant payé  
record_id                            Date                                
2927e24b-cd71-444c-b015-b713cd23aeab 2022-03-17   120,00 $          NaN  
                                     2022-03-16        NaN     120,00 $  
                                     2022-03-15        NaN     120,00 $  
                                     2022-03-14        NaN     120,00 $  
                                     2022-03-07        NaN       0,00 $  
...                                                    ...          ...  
1fe02096-dd22-435a-9cdb-b4877d933f12 2022-03-14        NaN     100,00 $  
                                     2022-03-14        NaN     100,00 $  
                                     2022-03-14        NaN     100,00 $  
                                     2022-03-14        NaN     100,00 $  
                                     2022-03-13        NaN     100,00 $  

[67 rows x 6 columns]

In [246]:
def create_missing_invoices(session, appoints_df, invoices_df):
    invoices_df = appoints_df.join(invoices_df, on=['record_id', 'Date'])
    invoices_df = invoices_df.reset_index()
    
    invoices_df.sort_values(['Date', 'Heure début'], inplace=True)
    missing_invoices = invoices_df.loc[invoices_df['Facture'].isna()]
    
    for i, invoice in missing_invoices.iterrows():
        create_invoice(session, invoice)


def create_invoice(session, invoice):
    record_id = invoice['record_id']
    endpoint = f'{base_url}/{record_id}/invoices'
    
    resp = session.get(f'{endpoint}/create')
    soup = BeautifulSoup(resp.content, 'html.parser')
    forms = soup.find_all('form')
    
    client_names = ' et '.join(filter(bool, invoice[['Client 1', 'Client 2']]))

    payload = {}

    for field in forms[1].find_all('input'):
        field_name, field_value = field.get('name'), field.get('value', '')
        if field_name is not None:
            add_to_payload(payload, field_name, field_value)

    for field in forms[1].find_all('textarea'):
        field_name, field_value = field.get('name'), field.get_text()
        add_to_payload(payload, field_name, field_value)

    payload['institution'] = {'id': ''}
    # payload['paymentDate'] = str(datetime.today()).split()[0]
    # payload['paymentTypes'] = ','.join(['debit_transfer'])
    payload['paymentDate'] = ''
    payload['paymentTypes'] = ''
    payload['meta']['charged_at'] = invoice['Date']
    payload['meta']['client_name'] = client_names
    payload['meta']['billed_to_name'] = client_names
    payload['items']['0'].update({
        'service': 'Suivi client',
        'price_per_unit': payload['items']['0']['price_per_unit'].replace(',', '.'),
    })
    
    resp = session.post(endpoint, data=json.dumps(payload))
    logger.info(f'ended with status code: {resp.status_code}')

    # open_html(resp.content)

        
def add_to_payload(payload, field_name, field_value, pattern=r'(\[|\]|\]\[])'):
    
    def pairwise(iterable):
        # pairwise('ABCDEFG') --> AB BC CD DE EF FG
        a, b = tee(iterable)
        next(b, None)
        return zip(a, b)

    keys = re.sub(pattern, ' ', field_name).split()
    if len(keys) == 1:
        payload[field_name] = field_value
    else:
        current = payload
        for k1, k2 in pairwise(keys):
            if k1 in current:
                current = current[k1]
            else:
                current[k1] = {}
                current = current[k1]

        current[k2] = field_value

In [249]:
def get_unpaid_invoices(session):
    dfs = []
    page = 1

    columns = {
        'record_id': 'record_id',
        'Facturé le': 'Date',
        'Facture': 'Facture',
        'Service(s)': 'Service(s)',
        'Montant dû': 'Montant dû',
        'invoice_id': 'invoice_id',
    }

    while True:
        # fetch next page
        resp = session.get(f'https://admin.psylio.com/invoices?page={page}')
        
        # extract invoices from html
        df = pd.read_html(resp.text)
        
        # break if page doesn't contain any invoice
        if len(df) < 2:
            break

        # parse page html
        soup = BeautifulSoup(resp.content, 'html.parser')

        # for each invoice, extract invoice link
        links = soup.find_all('a', {'data-target': '#mark-as-paid-modal'})
        links = [link.get('data-route') for link in links]
        
        # extract record_id and invoice_id for each invoice
        links = [link.split('/') for link in links]
        record_ids = [link[4] for link in links]
        invoice_ids = [link[6] for link in links]

        # update dataframe
        df = df[1]
        df['record_id'] = record_ids
        df['invoice_id'] = invoice_ids

        # save dataframe and update page number
        dfs.append(df)
        page += 1

    
    # merge dataframes and reorder columns
    unpaid_df = pd.concat(dfs)
    unpaid_df = unpaid_df[list(columns.keys())]
    
    # rename columns and set index
    unpaid_df.rename(columns=columns, inplace=True)
    unpaid_df.set_index(['record_id', 'Date'], inplace=True)
    
    return unpaid_df

In [284]:
create_missing_invoices(session, appoints_df, invoices_df)
unpaid_df = get_unpaid_invoices(session)
unpaid_df

Facture  \
record_id                            Date                          
7d2510c7-1614-4c02-8102-b9c7cf8479d4 2022-03-14  INV-20220314-72   
                                     2022-03-14  INV-20220314-71   
                                     2022-03-14  INV-20220314-70   
                                     2022-03-14  INV-20220314-69   
                                     2022-03-14  INV-20220314-68   
                                     2022-03-14  INV-20220314-67   
                                     2022-03-14  INV-20220314-66   
                                     2022-03-14  INV-20220314-65   
                                     2022-03-14  INV-20220314-62   
                                     2022-03-14  INV-20220314-61   
                                     2022-03-14  INV-20220314-59   
                                     2022-03-14  INV-20220314-58   
                                     2022-03-14  INV-20220314-57   
                                     2022-03-14  INV-20220314-56   
                                     2022-03-14  INV-20220314-55   
                                     2022-03-14  INV-20220314-54   
                                     2022-03-14  INV-20220314-53   
                                     2022-03-14  INV-20220314-52   
1fe02096-dd22-435a-9cdb-b4877d933f12 2022-03-12  INV-20220312-14   
                                     2022-03-12  INV-20220312-13   
                                     2022-03-12  INV-20220312-12   
                                     2022-03-12   INV-20220312-9   
7d2510c7-1614-4c02-8102-b9c7cf8479d4 2022-03-11  INV-20220314-76   
                                     2022-03-10  INV-20220314-75   
                                     2022-03-09  INV-20220314-73   
                                     2022-03-08  INV-20220314-74   

                                                   Service(s) Montant dû  \
record_id                            Date                                  
7d2510c7-1614-4c02-8102-b9c7cf8479d4 2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
                                     2022-03-14  Suivi client   110,00 $   
1fe02096-dd22-435a-9cdb-b4877d933f12 2022-03-12  Suivi client   100,00 $   
                                     2022-03-12  Suivi client   100,00 $   
                                     2022-03-12  Suivi client   100,00 $   
                                     2022-03-12  Suivi client   100,00 $   
7d2510c7-1614-4c02-8102-b9c7cf8479d4 2022-03-11  Suivi client   110,00 $   
                                     2022-03-10  Suivi client   110,00 $   
                                     2022-03-09  Suivi client   110,00 $   
                                     2022-03-08  Suivi client   110,00 $   

                       

In [277]:
def close_invoice(driver, invoice):
    # open invoice page
    record_id, invoice_id = invoice[['record_id', 'invoice_id']]
    base_url = 'https://admin.psylio.com/assistance-requests'
    invoice_url = f'{base_url}/{record_id}/invoices/{invoice_id}'
    driver.get(invoice_url)
    
    # get invoice infos
    payment_date = invoice['Date paiement']
    payment_types = 'debit_transfer' if invoice['Interac'] else 'cash'
    # payment_date, payment_types = invoice[['Date paiement', 'paymentTypes']]
    
    try:
        # mark invoice as paid
        mark_invoice_as_paid(driver, payment_date, payment_types)
        time.sleep(2)
    except Exception:
        pass
    
    try:
        # send invoice receipt
        # emails = ', '.join(filter(bool, invoice[['Courriel 1', 'Courriel 2']]))
        send_invoice_receipt(driver, invoice_url)
    except Exception:
        pass
    

def mark_invoice_as_paid(driver, payment_date, payment_types):
    # open the form
    mark_as_paid_btn = driver.find_element(By.XPATH, '//a[@data-target="#mark-as-paid-modal"]')
    mark_as_paid_btn.click()
    time.sleep(1)
    
    # select the form
    mark_as_paid_form = driver.find_element(By.XPATH, f'//form[@class="paymentType"]')
    
    # set payment date
    date_field = mark_as_paid_form.find_element(By.XPATH, '//input[@name="paymentDate"]')
    driver.execute_script("arguments[0].value = ''", date_field)
    date_field.send_keys(str(payment_date))
    date_field.send_keys(Keys.RETURN)
    
    # set payment type
    type_field = mark_as_paid_form.find_element(By.XPATH, f'//input[@name="paymentTypes[{payment_types}]"]')
    type_field.click()
    
    # submit the form
    mark_as_paid_form.submit()
    
    
def send_invoice_receipt(driver, invoice_url):
    # open the form
    send_receipt_btn = driver.find_element(By.XPATH, '//a[@data-target="#send-receipt-by-email-modal"]')
    send_receipt_btn.click()
    time.sleep(1)
    
    # select and submit the form
    send_receipt_form = driver.find_element(By.XPATH, f'//form[@action="{invoice_url}/receipt/email"]')
    send_receipt_form.submit()

# Main Program

In [ ]:
# records_df = get_records(session)
# record_id = records_df.iloc[0]['record_id']
# create_invoice(session, record_id)
# receipts_df = get_receipts(session, record_id)
# receipt_id = receipts_df.loc[0]['receipt_id']
# send_receipt(session, record_id, invoice_id)

In [ ]:
output_dir = './output'
os.makedirs(output_dir, exist_ok=True) 

records_df = get_records(session)
records_df.to_csv(f'{output_dir}/records.csv')
appoints_df = get_appointments(session, records_df)
appoints_df.to_csv(f'{output_dir}/appoints.csv')
invoices_df = create_invoices(session, appoints_df)
invoices_df.to_csv(f'{output_dir}/invoices.csv')

not_paid = invoices_df.copy()
not_paid = not_paid.loc[not_paid['État'].isin(['Brouillon', 'Draft'])]

not_paid['paymentDate'] = ''
not_paid['debit_transfer'] = ''
not_paid['cash'] = ''


# not_paid['paymentDate'] = '2022-03-15'
# not_paid['paymentTypes'] = 'debit_transfer' # ['debit_transfer', 'cash']

not_paid.to_csv(f'{output_dir}/not_paid.csv')


# endpoint = 'https://admin.psylio.com/assistance-requests'

# for i, invoice in not_paid.iterrows():
#     # get invoice infos
#     record_id, invoice_id = invoice[['record_id', 'invoice_id']]
#     payment_date, payment_types = invoice[['paymentDate', 'paymentTypes']]
    
#     # mark invoice as paid
#     invoice_url = f'{endpoint}/{record_id}/invoices/{invoice_id}'
#     driver.get(invoice_url)
    
#     # mark invoice as paid and send receipt
#     mark_invoice_as_paid(driver, payment_date, payment_types)
#     time.sleep(2)
#     send_invoice_receipt(driver)

In [111]:
# updated_df = pd.read_csv(filename)#.fillna('')
# updated_df['Date paiement'] = pd.to_datetime(updated_df['Date paiement'])
# updated_df

,Facture,Date,Numéro de dossier,Client 1,Client 2,Service(s),Montant dû,Date paiement,Comptant,Interac
0,INV-20220316-84,2022-03-16,150,"blo, mymy","blu, ladu",Suivi client,"120,00 $",2022-03-17,x,NaN
1,INV-20220317-87,2022-03-16,170,"mou, caca",NaN,Suivi client,"110,00 $",2022-03-17,NaN,x
2,INV-20220317-88,2022-03-17,150,"blo, mymy","blu, ladu",Suivi client,"120,00 $",NaT,NaN,NaN
3,INV-20220317-86,2022-03-17,170,"mou, caca",NaN,Suivi client,"110,00 $",NaT,NaN,NaN


In [112]:
# has_date = updated_df['Date paiement'].notna()
# is_cash = updated_df['Comptant'].notna()
# is_debit = updated_df['Interac'].notna()


# to_mark = has_date & (is_cash ^ is_debit)
# to_drop = ~(has_date | is_cash | is_debit)
# invalid = ~(to_mark ^ to_drop)

# if any(invalid):
#     print("ERROR!")
# else:
#     to_mark_as_paid_df = updated_df.loc[updated_df['Date paiement'].notna()]

#     with pd.option_context('mode.chained_assignment', None):
#         for col in ['Comptant', 'Interac']:
#             to_mark_as_paid_df[col] = to_mark_as_paid_df[col].str.strip()

#     to_mark_as_paid_df

,Facture,Date,Numéro de dossier,Client 1,Client 2,Service(s),Montant dû,Date paiement,Comptant,Interac
0,INV-20220316-84,2022-03-16,150,"blo, mymy","blu, ladu",Suivi client,"120,00 $",2022-03-17,x,NaN
1,INV-20220317-87,2022-03-16,170,"mou, caca",NaN,Suivi client,"110,00 $",2022-03-17,NaN,x


In [192]:
# newly_paid_df.set_index('Facture', inplace=True)
# unpaid_df.set_index('Facture', inplace=True)

In [285]:
def write_unpaid_to_file(records_df, unpaid_df, filename):
    unpaid_df['paymentDate'] = ''
    unpaid_df['debit_transfer'] = ''
    unpaid_df['cash'] = ''

    unpaid_df = unpaid_df.join(records_df, on='record_id')
    unpaid_df.reset_index(inplace=True)


    columns = ['Facture', 'Date', 'Numéro de dossier', 'Client 1', 'Client 2', 'Service(s)', 'Montant dû']
    to_file_df = unpaid_df[columns].sort_values('Date')

    to_file_df.set_index(['Facture', 'Date'], inplace=True)
    to_file_df[['Date paiement', 'Comptant', 'Interac']] = ''
    to_file_df.to_csv(filename)

    if os.name == 'nt':
        os.startfile(filename).read()
    else:
        os.popen(f'libreoffice --calc {filename}').read()

In [290]:
filename = 'unpaid.csv'
write_unpaid_to_file(records_df, unpaid_df, filename)

In [291]:
with_paid_df = pd.read_csv(filename)
with_paid_df

,Facture,Date,Numéro de dossier,Client 1,Client 2,Service(s),Montant dû,Date paiement,Comptant,Interac
0,INV-20220314-74,2022-03-08,160,"toi, salut",NaN,Suivi client,"110,00 $",NaN,NaN,NaN
1,INV-20220314-73,2022-03-09,160,"toi, salut",NaN,Suivi client,"110,00 $",NaN,NaN,NaN
2,INV-20220314-75,2022-03-10,160,"toi, salut",NaN,Suivi client,"110,00 $",NaN,NaN,NaN
3,INV-20220314-76,2022-03-11,160,"toi, salut",NaN,Suivi client,"110,00 $",NaN,NaN,NaN
4,INV-20220312-9,2022-03-12,111,"bli#, blu#",NaN,Suivi client,"100,00 $",NaN,NaN,NaN
5,INV-20220312-12,2022-03-12,111,"bli#, blu#",NaN,Suivi client,"100,00 $",NaN,NaN,NaN
6,INV-20220312-13,2022-03-12,111,"bli#, blu#",NaN,Suivi client,"100,00 $",NaN,NaN,NaN
7,INV-20220312-14,2022-03-12,111,"bli#, blu#",NaN,Suivi client,"100,00 $",NaN,NaN,NaN
8,INV-20220314-52,2022-03-14,160,"toi, salut",NaN,Suivi client,"110,00 $",NaN,NaN,NaN
9,INV-20220314-53,2022-03-14,160,"toi, salut",NaN,Suivi client,"110,00 $",NaN,NaN,NaN


In [299]:
def get_newly_paid(with_paid_df, filename):
    with_paid_df = pd.read_csv(filename)
    with_paid_df['Date paiement'] = pd.to_datetime(with_paid_df['Date paiement'])
    with_paid_df

    has_date = with_paid_df['Date paiement'].notna()
    is_cash = with_paid_df['Comptant'].notna()
    is_debit = with_paid_df['Interac'].notna()


    to_mark = has_date & (is_cash ^ is_debit)
    to_drop = ~(has_date | is_cash | is_debit)
    invalid = ~(to_mark ^ to_drop)

    assert not any(invalid), 'Inconsistent values have been found!'

    newly_paid_df = with_paid_df.loc[with_paid_df['Date paiement'].notna()]

    with pd.option_context('mode.chained_assignment', None):
        for col in ['Comptant', 'Interac']:
            if newly_paid_df[col].dtype in ('object', 'string'):
                newly_paid_df[col] = newly_paid_df[col].str.strip()

    return newly_paid_df

In [300]:
newly_paid_df = get_newly_paid(with_paid_df, filename)
newly_paid_df

,Facture,Date,Numéro de dossier,Client 1,Client 2,Service(s),Montant dû,Date paiement,Comptant,Interac
25,INV-20220314-57,2022-03-14,160,"toi, salut",NaN,Suivi client,"110,00 $",2022-03-17,x,NaN


In [257]:
newly_paid_df.set_index('Facture', inplace=True)
unpaid_df.set_index('Facture', inplace=True)

In [279]:
def close_paid_invoices(email, password, unpaid_df, newly_paid_df):
    # instantiate driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get('https://admin.psylio.com')

    login_form = driver.find_element(
        By.XPATH, '//form[@action="https://admin.psylio.com/login"]')

    email_field = login_form.find_element(
        By.XPATH, '//input[@name="login[email]"]')
    password_field = login_form.find_element(
        By.XPATH, '//input[@name="login[password]"]')

    email_field.send_keys(email)
    password_field.send_keys(password)

    login_form.submit()

    # get newly paid invoices
    columns = ['Date paiement', 'Comptant', 'Interac']
    to_finalize_df = newly_paid_df[columns].join(unpaid_df, on='Facture')

    # mark invoices as paid and send receipts
    for _, invoice in to_finalize_df.iterrows():
        close_invoice(driver, invoice)

    time.sleep(3)
    driver.quit()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/home/francis/.wdm/drivers/chromedriver/linux64/99.0.4844.51/chromedriver] found in cache


In [ ]:
close_paid_invoices(email, password, unpaid_df, newly_paid_df)